In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cudf as cd
import cupy
import time

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda ff: '%.2f' % ff)

chfs = {2011: {}, 2013: {}, 2015: {}, 2017: {}, 2019: {}}
cupy.cuda.Device(2).use()

ModuleNotFoundError: No module named 'cudf'

In [2]:
t = time.time()
    
def te():
    print("spent",(time.time() - t), "s")

NameError: name 'time' is not defined

In [1]:

# 计算除工资收入之外的其他

'''
计算其他的个人收入
转移性收入的中的个人收入
2019:
    养老金 f1005
    个人月均年金 f1028
    个人住房公积金收入 f4011
    保险 f4011 f6119 f6204 f6503 f3004a_imp f3003a_imp

2017:
    养老金 f1005
    月均年金 f1028
    个人住房公积金 f4011
    保险 1019 f2025 f6119 f6204 f6503 

2015:
    
    
2013 

2011：没有养老金
'''
#  自己拍拍脑子给出的一个可解释性：这些转移性收入随时间的变化没有labor收入变化剧烈






'\n计算其他的个人收入\n转移性收入的中的个人收入\n2019:\n    养老金 f1005\n    个人月均年金 f1028\n    个人住房公积金收入 f4011\n    保险 f4011 f6119 f6204 f6503 f3004a_imp f3003a_imp\n\n2017:\n    养老金 f1005\n    月均年金 f1028\n    个人住房公积金 f4011\n    保险 1019 f2025 f6119 f6204 f6503 \n\n2015:\n    \n    \n   \n'

In [ ]:
'''
    加入其他一些特征
        家庭成员所在地区
        职业
        补贴：

'''

# 2011年数据

In [3]:
# #预处理 dta转csv
# pd.read_stata("./chfs2011/ind.dta").fillna(0).to_csv("./chfs2011/ind.csv",index=False)
# pd.read_stata("./chfs2011/hh.dta").fillna(0).to_csv("./chfs2011/hh.csv",index=False)
# pd.read_stata("./chfs2011/master.dta").fillna(0).to_csv("./chfs2011/master.csv",index=False)

## 读取数据

In [4]:
chfs[2011]['ind'] = cd.read_csv("./chfs2011/ind.csv")
chfs[2011]['ind'].shape

(29324, 251)

In [5]:
chfs[2011]['hh'] = cd.read_csv("./chfs2011/hh.csv")
chfs[2011]['hh'].shape

(8438, 1272)

## 拼接两个表

In [6]:
chfs[2011]['ind_hh'] = cd.merge(chfs[2011]['ind'], chfs[2011]['hh'], how='left', on='hhid')
del chfs[2011]['ind']
del chfs[2011]['hh']
chfs[2011]['ind_hh'].shape

(29324, 1522)

## 计算各种各样的收入

## 计算 工资性收入，家庭成员，

In [7]:
# labor_inc 是指个人税后工资薪金收入，即为劳动性收入(包括调查问卷中提到的最主要的工作以及第二职业工作税后工资薪金收入总和)。
chfs[2011]['ind_hh'] = chfs[2011]['ind_hh'].assign(age=2011-chfs[2011]['ind_hh'].a2005) #计算age

dict_mem_cnt = chfs[2011]['ind_hh'].groupby('hhid').size().to_pandas().to_dict() #计算家庭成员个数
chfs[2011]['ind_hh']['mem_cnt'] = chfs[2011]['ind_hh']['hhid'].map(dict_mem_cnt)

dict_labor_inc = chfs[2011]['ind_hh'].groupby('hhid')['labor_inc'].agg('sum').to_pandas().to_dict() #计算家庭总labor收入
chfs[2011]['ind_hh']['labor_inc_sum'] = chfs[2011]['ind_hh']['hhid'].map(dict_labor_inc)

dict_labor_inc_mem_cnt = chfs[2011]['ind_hh'].groupby('hhid')['labor_inc'].count().to_pandas().to_dict() #计算家庭有labor收入的人数
chfs[2011]['ind_hh']['labor_inc_mem_cnt'] = chfs[2011]['ind_hh']['hhid'].map(dict_labor_inc_mem_cnt)

#性别
chfs[2011]['ind_hh']['male'] = chfs[2011]['ind_hh']['a2003'].map({1: 1}).fillna(0) #男
chfs[2011]['ind_hh']['female'] = chfs[2011]['ind_hh']['a2003'].map({2: 1}).fillna(0) #女

#学历
chfs[2011]['ind_hh']['el_non'] = chfs[2011]['ind_hh']['a2012'].map({1: 1}).fillna(0) #没上过学
chfs[2011]['ind_hh']['el_pri'] = chfs[2011]['ind_hh']['a2012'].map({2: 1}).fillna(0) #小学
chfs[2011]['ind_hh']['el_jun'] = chfs[2011]['ind_hh']['a2012'].map({3: 1}).fillna(0) #初中
chfs[2011]['ind_hh']['el_sen'] = chfs[2011]['ind_hh']['a2012'].map({4: 1}).fillna(0) #高中
chfs[2011]['ind_hh']['el_ts'] = chfs[2011]['ind_hh']['a2012'].map({5: 1}).fillna(0) #中专
chfs[2011]['ind_hh']['el_jc'] = chfs[2011]['ind_hh']['a2012'].map({6: 1}).fillna(0) #大专
chfs[2011]['ind_hh']['el_bac'] = chfs[2011]['ind_hh']['a2012'].map({7: 1}).fillna(0) #本科
chfs[2011]['ind_hh']['el_mas'] = chfs[2011]['ind_hh']['a2012'].map({8: 1}).fillna(0) #硕士
chfs[2011]['ind_hh']['el_phd'] = chfs[2011]['ind_hh']['a2012'].map({9: 1}).fillna(0) #博士

#车
chfs[2011]['ind_hh']['c_car'] = chfs[2011]['ind_hh']['c7001'].map({1: 1}).fillna(0) #汽车
chfs[2011]['ind_hh']['c_car_n'] = chfs[2011]['ind_hh']['c7002'] #汽车数量
chfs[2011]['ind_hh']['c_car2'] = chfs[2011]['ind_hh']['c7053'].map({1: 1}).fillna(0) #拖拉机等


## 处理数据column成统一可读性列名

In [8]:
#c8001: 相机1、电视2、洗衣机3、冰箱4、空调5、电脑6、音响7、热水器8、家具9、卫星接收器10、乐器11
chfs[2011]['ind_hh']['c_cam'] = chfs[2011]['ind_hh']['c8001_1_mc'] 
chfs[2011]['ind_hh']['c_tv'] = chfs[2011]['ind_hh']['c8001_2_mc'] 
chfs[2011]['ind_hh']['c_wm'] = chfs[2011]['ind_hh']['c8001_3_mc'] 
chfs[2011]['ind_hh']['c_ref'] = chfs[2011]['ind_hh']['c8001_4_mc'] 
chfs[2011]['ind_hh']['c_ac'] = chfs[2011]['ind_hh']['c8001_5_mc'] 
chfs[2011]['ind_hh']['c_cp'] = chfs[2011]['ind_hh']['c8001_6_mc'] 
chfs[2011]['ind_hh']['c_aud'] = chfs[2011]['ind_hh']['c8001_7_mc'] 
chfs[2011]['ind_hh']['c_wh'] = chfs[2011]['ind_hh']['c8001_8_mc'] 
chfs[2011]['ind_hh']['c_fur'] = chfs[2011]['ind_hh']['c8001_9_mc'] 
chfs[2011]['ind_hh']['c_sr'] = chfs[2011]['ind_hh']['c8001_10_mc'] 
chfs[2011]['ind_hh']['c_mi'] = chfs[2011]['ind_hh']['c8001_11_mc'] 

#c8003: 其它资产如飞机、金银首饰 c8004_5_mc: 金银首饰
chfs[2011]['ind_hh']['c_oa'] = chfs[2011]['ind_hh']['c8003'].map({1: 1}).fillna(0)
chfs[2011]['ind_hh']['c_je'] = chfs[2011]['ind_hh']['c8004_5_mc']

#吃饭消费：g1001 月*12
chfs[2011]['ind_hh']['g_eat'] = chfs[2011]['ind_hh']['g1001']*12
#日用品支出：g1006 月*12
chfs[2011]['ind_hh']['g_dn'] = chfs[2011]['ind_hh']['g1006']*12
#雇佣保姆小时工等支出：g1007 月*12
chfs[2011]['ind_hh']['g_hw'] = chfs[2011]['ind_hh']['g1007']*12
#本地交通费：g1008 月*12
chfs[2011]['ind_hh']['g_tp'] = chfs[2011]['ind_hh']['g1008']*12
#电话网络通信费：g1009 月*12
chfs[2011]['ind_hh']['g_comm'] = chfs[2011]['ind_hh']['g1009']*12
#文化娱乐：g1010 月*12
chfs[2011]['ind_hh']['g_et'] = chfs[2011]['ind_hh']['g1010']*12
#水电燃气+暖气：g1005月*12+g1013年
chfs[2011]['ind_hh'] = chfs[2011]['ind_hh'].assign(g_sdrm=chfs[2011]['ind_hh']['g1005']*12+chfs[2011]['ind_hh']['g1013'])
#买衣服：g1011年+g1011a年+g1011b年
chfs[2011]['ind_hh'] = chfs[2011]['ind_hh'].assign(g_clo=chfs[2011]['ind_hh']['g1011']+chfs[2011]['ind_hh']['g1011a']+chfs[2011]['ind_hh']['g1011b'])
#教育培训支出：g1016 年
chfs[2011]['ind_hh']['g_edu'] = chfs[2011]['ind_hh']['g1016']
#旅游探亲支出：g1018_imp 年
chfs[2011]['ind_hh']['g_trav'] = chfs[2011]['ind_hh']['g1018_imp']
#仅保健支出：g1019 年
chfs[2011]['ind_hh']['g_med'] = chfs[2011]['ind_hh']['g1019']

#d1105_imp: 活期存款 d2104_imp: 定期存款
chfs[2011]['ind_hh']['d_dd'] = chfs[2011]['ind_hh']['d1105_imp']
chfs[2011]['ind_hh']['d_fd'] = chfs[2011]['ind_hh']['d2104_imp']
#债券：d4103_(1~5)_imp
for i in range(1,6):
    chfs[2011]['ind_hh']['d_bond_'+str(i)] = chfs[2011]['ind_hh']['d4103_'+str(i)+'_imp']
#债券总
chfs[2011]['ind_hh'] = chfs[2011]['ind_hh'].assign(d_bond=chfs[2011]['ind_hh']['d_bond_1']+chfs[2011]['ind_hh']['d_bond_2']+chfs[2011]['ind_hh']['d_bond_3']+chfs[2011]['ind_hh']['d_bond_4']+chfs[2011]['ind_hh']['d_bond_5'])
#银行理财/互联网理财：d7106a_imp
chfs[2011]['ind_hh']['d_fm'] = chfs[2011]['ind_hh']['d7106a_imp']
#黄金/贵金属：d9103_imp
chfs[2011]['ind_hh']['d_gold'] = chfs[2011]['ind_hh']['d9103_imp']
#现金：k1101_imp
chfs[2011]['ind_hh']['d_cash'] = chfs[2011]['ind_hh']['k1101_imp']

chfs[2011]['ind_hh']['total_income'] = chfs[2011]['ind_hh']['hh_income']

chfs[2011]['ind_hh']['year'] = 2011

## 输出表

In [9]:
chfs[2011]['ind_hh'][['hhid','year','age','male','female', \
                      'el_non','el_pri','el_jun','el_sen','el_ts','el_jc','el_bac','el_mas','el_phd', \
                      'c_car','c_car_n','c_car2', \
                      'c_cam','c_tv','c_wm','c_ref','c_ac','c_cp','c_aud','c_wh','c_fur','c_sr','c_mi', \
                      'c_oa','c_je', \
                      'd_dd','d_fd','d_bond','d_fm','d_gold','d_cash', \
                      'g_eat','g_sdrm','g_dn','g_hw','g_tp','g_comm','g_et', \
                      'g_clo','g_edu','g_trav','g_med', \
                      'mem_cnt','labor_inc','labor_inc_sum','labor_inc_mem_cnt','total_income']]
#                       'mem_cnt','labor_inc','labor_inc_sum','labor_inc_mem_cnt','total_income']].to_csv("./chfs2011/final.csv",index=False)

# 2013年数据

In [7]:
# #预处理 dta转csv
# pd.read_stata("./chfs2013/ind.dta").fillna(0).to_csv("./chfs2013/ind.csv",index=False)
# pd.read_stata("./chfs2013/hh.dta").fillna(0).to_csv("./chfs2013/hh.csv",index=False)
# pd.read_stata("./chfs2013/master.dta").fillna(0).to_csv("./chfs2013/master.csv",index=False)

## 读取数据

In [10]:
chfs[2013]['ind'] = pd.read_stata("./chfs2013/ind.dta").fillna(0)
chfs[2013]['ind'].shape

(97906, 332)

In [11]:
chfs[2013]['hh'] = pd.read_stata("./chfs2013/hh.dta").fillna(0)
chfs[2013]['hh'].shape

(28141, 2184)

In [12]:
def get_hhid(x):
    if x['hhid_2011'] =='':
        return x['hhid_2013']
    else:
        return x['hhid_2011']

#填充hhid，因为2013年新数据没有hhid_2011的列，所以整合一下统一使用hhid
chfs[2013]['ind'].loc[:,'hhid'] = chfs[2013]['ind'].apply(get_hhid, axis=1)
chfs[2013]['ind'] = cd.from_pandas(chfs[2013]['ind'])
chfs[2013]['hh'].loc[:,'hhid'] = chfs[2013]['hh'].apply(get_hhid, axis=1)
chfs[2013]['hh'] = cd.from_pandas(chfs[2013]['hh'])

## 拼接两个表

In [13]:
chfs[2013]['ind_hh'] = cd.merge(chfs[2013]['ind'], chfs[2013]['hh'], how='left', on='hhid')
del chfs[2013]['ind']
del chfs[2013]['hh']
chfs[2013]['ind_hh'].shape

(97906, 2517)

## 计算表中未有的参数值

In [14]:
chfs[2013]['ind_hh'] = chfs[2013]['ind_hh'].assign(age=2013-chfs[2013]['ind_hh'].a2005) #计算age

dict_mem_cnt = chfs[2013]['ind_hh'].groupby('hhid').size().to_pandas().to_dict() #计算家庭成员个数
chfs[2013]['ind_hh']['mem_cnt'] = chfs[2013]['ind_hh']['hhid'].map(dict_mem_cnt)

chfs[2013]['ind_hh']['labor_inc'] = chfs[2013]['ind_hh']['a3020_imp']+chfs[2013]['ind_hh']['a3022_imp']+chfs[2013]['ind_hh']['a3023_imp']+chfs[2013]['ind_hh']['a3036_imp']
dict_labor_inc = chfs[2013]['ind_hh'].groupby('hhid')['labor_inc'].agg('sum').to_pandas().to_dict() #计算家庭总labor收入
chfs[2013]['ind_hh']['labor_inc_sum'] = chfs[2013]['ind_hh']['hhid'].map(dict_labor_inc)

dict_labor_inc_mem_cnt = chfs[2013]['ind_hh'].groupby('hhid')['labor_inc'].count().to_pandas().to_dict() #计算家庭有labor收入的人数
chfs[2013]['ind_hh']['labor_inc_mem_cnt'] = chfs[2013]['ind_hh']['hhid'].map(dict_labor_inc_mem_cnt)

#性别
chfs[2013]['ind_hh']['male'] = chfs[2013]['ind_hh']['a2003'].map({1: 1}).fillna(0) #男
chfs[2013]['ind_hh']['female'] = chfs[2013]['ind_hh']['a2003'].map({2: 1}).fillna(0) #女

#学历
chfs[2013]['ind_hh']['el_non'] = chfs[2013]['ind_hh']['a2012'].map({1: 1}).fillna(0) #没上过学
chfs[2013]['ind_hh']['el_pri'] = chfs[2013]['ind_hh']['a2012'].map({2: 1}).fillna(0) #小学
chfs[2013]['ind_hh']['el_jun'] = chfs[2013]['ind_hh']['a2012'].map({3: 1}).fillna(0) #初中
chfs[2013]['ind_hh']['el_sen'] = chfs[2013]['ind_hh']['a2012'].map({4: 1}).fillna(0) #高中
chfs[2013]['ind_hh']['el_ts'] = chfs[2013]['ind_hh']['a2012'].map({5: 1}).fillna(0) #中专 
chfs[2013]['ind_hh']['el_jc'] = chfs[2013]['ind_hh']['a2012'].map({6: 1}).fillna(0) #大专
chfs[2013]['ind_hh']['el_bac'] = chfs[2013]['ind_hh']['a2012'].map({7: 1}).fillna(0) #本科
chfs[2013]['ind_hh']['el_mas'] = chfs[2013]['ind_hh']['a2012'].map({8: 1}).fillna(0) #硕士
chfs[2013]['ind_hh']['el_phd'] = chfs[2013]['ind_hh']['a2012'].map({9: 1}).fillna(0) #博士

#车
chfs[2013]['ind_hh']['c_car'] = chfs[2013]['ind_hh']['c7001'].map({1: 1}).fillna(0) #汽车
chfs[2013]['ind_hh']['c_car_n'] = chfs[2013]['ind_hh']['c7002'] #汽车数量
chfs[2013]['ind_hh']['c_car2'] = chfs[2013]['ind_hh']['c7058'].map({0: 0}).fillna(1) #电瓶车等，c7058判断

chfs[2013]['ind_hh']['year'] = 2013

## 处理数据column成统一可读性列名

In [15]:
#c8001: 相机1、电视2、洗衣机3、冰箱4、空调5、电脑6、音响7、热水器8、家具9、卫星接收器10、乐器11、手机12
chfs[2013]['ind_hh']['c_cam'] = chfs[2013]['ind_hh']['c8001_1_mc'] 
chfs[2013]['ind_hh']['c_tv'] = chfs[2013]['ind_hh']['c8001_2_mc'] 
chfs[2013]['ind_hh']['c_wm'] = chfs[2013]['ind_hh']['c8001_3_mc'] 
chfs[2013]['ind_hh']['c_ref'] = chfs[2013]['ind_hh']['c8001_4_mc'] 
chfs[2013]['ind_hh']['c_ac'] = chfs[2013]['ind_hh']['c8001_5_mc'] 
chfs[2013]['ind_hh']['c_cp'] = chfs[2013]['ind_hh']['c8001_6_mc'] 
chfs[2013]['ind_hh']['c_aud'] = chfs[2013]['ind_hh']['c8001_7_mc'] 
chfs[2013]['ind_hh']['c_wh'] = chfs[2013]['ind_hh']['c8001_8_mc'] 
chfs[2013]['ind_hh']['c_fur'] = chfs[2013]['ind_hh']['c8001_9_mc'] 
chfs[2013]['ind_hh']['c_sr'] = chfs[2013]['ind_hh']['c8001_10_mc'] 
chfs[2013]['ind_hh']['c_mi'] = chfs[2013]['ind_hh']['c8001_11_mc'] 
chfs[2013]['ind_hh']['c_mobi'] = chfs[2013]['ind_hh']['c8001_12_mc'] 

#c8003: 其它资产如飞机、金银首饰 c8004_5_mc: 金银首饰
chfs[2013]['ind_hh']['c_oa'] = chfs[2013]['ind_hh']['c8003'].map({1: 1}).fillna(0)
chfs[2013]['ind_hh']['c_je'] = chfs[2013]['ind_hh']['c8004_5_mc']

#吃饭消费：g1001_imp 月*12
chfs[2013]['ind_hh']['g_eat'] = chfs[2013]['ind_hh']['g1001_imp']*12
#日用品支出：g1006_imp 月*12
chfs[2013]['ind_hh']['g_dn'] = chfs[2013]['ind_hh']['g1006_imp']*12
#雇佣保姆小时工等支出：g1007_imp 月*12
chfs[2013]['ind_hh']['g_hw'] = chfs[2013]['ind_hh']['g1007_imp']*12
#本地交通费：g1008_imp 月*12
chfs[2013]['ind_hh']['g_tp'] = chfs[2013]['ind_hh']['g1008_imp']*12
#电话网络通信费：g1009_imp 月*12
chfs[2013]['ind_hh']['g_comm'] = chfs[2013]['ind_hh']['g1009_imp']*12
#文化娱乐：g1010_imp 月*12
chfs[2013]['ind_hh']['g_et'] = chfs[2013]['ind_hh']['g1010_imp']*12
#水电燃气+暖气：g1005_imp月*12+g1013_imp年
chfs[2013]['ind_hh'] = chfs[2013]['ind_hh'].assign(g_sdrm=chfs[2013]['ind_hh']['g1005_imp']*12+chfs[2013]['ind_hh']['g1013_imp'])
#买衣服：g1011_imp 年
chfs[2013]['ind_hh']['g_clo'] = chfs[2013]['ind_hh']['g1011_imp']
#教育培训支出：g1016_imp 年
chfs[2013]['ind_hh']['g_edu'] = chfs[2013]['ind_hh']['g1016_imp']
#旅游探亲支出：g1018_imp 年
chfs[2013]['ind_hh']['g_trav'] = chfs[2013]['ind_hh']['g1018_imp']
#仅保健支出：g1019_imp 年
chfs[2013]['ind_hh']['g_med'] = chfs[2013]['ind_hh']['g1019_imp']

#d1105_imp: 活期存款 d2104_imp: 定期存款
chfs[2013]['ind_hh']['d_dd'] = chfs[2013]['ind_hh']['d1105_imp']
chfs[2013]['ind_hh']['d_fd'] = chfs[2013]['ind_hh']['d2104_imp']
#债券：d4103_(1~5)_imp
for i in range(1,6):
    chfs[2013]['ind_hh']['d_bond_'+str(i)] = chfs[2013]['ind_hh']['d4103_'+str(i)+'_imp']
#债券总
chfs[2013]['ind_hh'] = chfs[2013]['ind_hh'].assign(d_bond=chfs[2013]['ind_hh']['d_bond_1']+chfs[2013]['ind_hh']['d_bond_2']+chfs[2013]['ind_hh']['d_bond_3']+chfs[2013]['ind_hh']['d_bond_4']+chfs[2013]['ind_hh']['d_bond_5'])
#银行理财/互联网理财：d7106a_imp
chfs[2013]['ind_hh']['d_fm'] = chfs[2013]['ind_hh']['d7106a_imp']
#黄金/贵金属：d9103_imp
chfs[2013]['ind_hh']['d_gold'] = chfs[2013]['ind_hh']['d9103_imp']
#现金：k1101_imp
chfs[2013]['ind_hh']['d_cash'] = chfs[2013]['ind_hh']['k1101_imp']

chfs[2013]['ind_hh']['total_income'] = chfs[2013]['ind_hh']['total_income_imp']

chfs[2013]['ind_hh']['year'] = 2013

## 输出表

In [16]:
chfs[2013]['ind_hh'][['hhid','year','age','male','female', \
                      'el_non','el_pri','el_jun','el_sen','el_ts','el_jc','el_bac','el_mas','el_phd', \
                      'c_car','c_car_n','c_car2', \
                      'c_cam','c_tv','c_wm','c_ref','c_ac','c_cp','c_aud','c_wh','c_fur','c_sr','c_mi','c_mobi', \
                      'c_oa','c_je', \
                      'd_dd','d_fd','d_bond','d_fm','d_gold','d_cash', \
                      'g_eat','g_sdrm','g_dn','g_hw','g_tp','g_comm','g_et', \
                      'g_clo','g_edu','g_trav','g_med', \
                      'mem_cnt','labor_inc','labor_inc_sum','labor_inc_mem_cnt','total_income']]
#                       'mem_cnt','labor_inc','labor_inc_sum','labor_inc_mem_cnt','total_income']].to_csv("./chfs2013/final.csv",index=False)

# 2015年数据

In [9]:
# #预处理 dta转csv
# pd.read_stata("./chfs2015/ind.dta").fillna(0).to_csv("./chfs2015/ind.csv",index=False)
# pd.read_stata("./chfs2015/hh.dta").fillna(0).to_csv("./chfs2015/hh.csv",index=False)
# pd.read_stata("./chfs2015/master.dta").fillna(0).to_csv("./chfs2015/master.csv",index=False)

## 读取数据

In [17]:
chfs[2015]['ind'] = pd.read_csv("./chfs2015/ind.csv", low_memory=False)
chfs[2015]['ind'].shape

(133183, 309)

In [18]:
chfs[2015]['hh'] = cd.read_csv("./chfs2015/hh.csv")
chfs[2015]['hh'].shape

(37289, 3011)

In [19]:
#2015需要计算插值 a3020:主要工作工资，a3022:主要工作奖金，a3023:主要工作补贴，a3036:第二份工作工资
row_names = ['a3020','a3022','a3023','a3036']
its = {'a3020': [5000,10000,20000,50000,100000,200000,300000,500000,1000000,2000000,5000000], 
       'a3022': [2500,5000,10000,20000,50000,100000,150000,200000,300000,500000,1000000],
       'a3023': [2500,5000,10000,20000,50000,100000,150000,200000,300000,500000,1000000],
       'a3036': [2500,5000,10000,20000,50000,100000,150000,200000,300000,500000,1000000]}
row_name = ''
def f_labor_inc_it(row):
    if row[row_name+'it'] == 0:
        return row[row_name]
    else:
        return its[row_name][int(row[row_name+'it'])-1]
for x in row_names:
    row_name = x
    chfs[2015]['ind']['labor_inc_'+x] = chfs[2015]['ind'].apply(f_labor_inc_it, axis=1)
# chfs[2015]['ind'][['a3020','a3020it','a3022','a3022it','a3023','a3023it','a3036','a3036it','labor_inc_a3020','labor_inc_a3022','labor_inc_a3023','labor_inc_a3036']][:60]

In [20]:
chfs[2015]['ind'] = cd.from_pandas(chfs[2015]['ind'])

## 拼接两个表

In [21]:
chfs[2015]['ind_hh'] = cd.merge(chfs[2015]['ind'], chfs[2015]['hh'], how='left', on='hhid')
chfs[2015]['ind_hh']
del chfs[2015]['ind']
del chfs[2015]['hh']
chfs[2015]['ind_hh'].shape

(133183, 3323)

#

## 计算表中未有的参数值

In [22]:
chfs[2015]['ind_hh'] = chfs[2015]['ind_hh'].assign(age=2015-chfs[2015]['ind_hh'].a2005) #计算age

dict_mem_cnt = chfs[2015]['ind_hh'].groupby('hhid').size().to_pandas().to_dict() #计算家庭成员个数
chfs[2015]['ind_hh']['mem_cnt'] = chfs[2015]['ind_hh']['hhid'].map(dict_mem_cnt)

chfs[2015]['ind_hh']['labor_inc'] = chfs[2015]['ind_hh']['labor_inc_a3020']+chfs[2015]['ind_hh']['labor_inc_a3022']+chfs[2015]['ind_hh']['labor_inc_a3023']+chfs[2015]['ind_hh']['labor_inc_a3036']
dict_labor_inc = chfs[2015]['ind_hh'].groupby('hhid')['labor_inc'].agg('sum').to_pandas().to_dict() #计算家庭总labor收入
chfs[2015]['ind_hh']['labor_inc_sum'] = chfs[2015]['ind_hh']['hhid'].map(dict_labor_inc)

dict_labor_inc_mem_cnt = chfs[2015]['ind_hh'].groupby('hhid')['labor_inc'].count().to_pandas().to_dict() #计算家庭有labor收入的人数
chfs[2015]['ind_hh']['labor_inc_mem_cnt'] = chfs[2015]['ind_hh']['hhid'].map(dict_labor_inc_mem_cnt)

#性别
chfs[2015]['ind_hh']['male'] = chfs[2015]['ind_hh']['a2003'].map({1: 1}).fillna(0) #男
chfs[2015]['ind_hh']['female'] = chfs[2015]['ind_hh']['a2003'].map({2: 1}).fillna(0) #女

#学历
chfs[2015]['ind_hh']['el_non'] = chfs[2015]['ind_hh']['a2012'].map({1: 1}).fillna(0) #没上过学
chfs[2015]['ind_hh']['el_pri'] = chfs[2015]['ind_hh']['a2012'].map({2: 1}).fillna(0) #小学
chfs[2015]['ind_hh']['el_jun'] = chfs[2015]['ind_hh']['a2012'].map({3: 1}).fillna(0) #初中
chfs[2015]['ind_hh']['el_sen'] = chfs[2015]['ind_hh']['a2012'].map({4: 1}).fillna(0) #高中
chfs[2015]['ind_hh']['el_ts'] = chfs[2015]['ind_hh']['a2012'].map({5: 1}).fillna(0) #中专 
chfs[2015]['ind_hh']['el_jc'] = chfs[2015]['ind_hh']['a2012'].map({6: 1}).fillna(0) #大专
chfs[2015]['ind_hh']['el_bac'] = chfs[2015]['ind_hh']['a2012'].map({7: 1}).fillna(0) #本科
chfs[2015]['ind_hh']['el_mas'] = chfs[2015]['ind_hh']['a2012'].map({8: 1}).fillna(0) #硕士
chfs[2015]['ind_hh']['el_phd'] = chfs[2015]['ind_hh']['a2012'].map({9: 1}).fillna(0) #博士

#车
chfs[2015]['ind_hh']['c_car'] = chfs[2015]['ind_hh']['c7001'].map({1: 1}).fillna(0) #汽车
chfs[2015]['ind_hh']['c_car_n'] = chfs[2015]['ind_hh']['c7002'] #汽车数量
chfs[2015]['ind_hh']['c_car2'] = chfs[2015]['ind_hh']['c7058'].map({0: 0}).fillna(1) #拖拉机等

chfs[2015]['ind_hh']['year'] = 2015

## 处理数据column成统一可读性列名

In [23]:
#c8001: 相机1、电视2、洗衣机3、冰箱4、空调5、电脑6、音响7、热水器8、家具9、卫星接收器10、乐器12、手机13
chfs[2015]['ind_hh']['c_cam'] = chfs[2015]['ind_hh']['c8001_1_mc'] 
chfs[2015]['ind_hh']['c_tv'] = chfs[2015]['ind_hh']['c8001_2_mc'] 
chfs[2015]['ind_hh']['c_wm'] = chfs[2015]['ind_hh']['c8001_3_mc'] 
chfs[2015]['ind_hh']['c_ref'] = chfs[2015]['ind_hh']['c8001_4_mc'] 
chfs[2015]['ind_hh']['c_ac'] = chfs[2015]['ind_hh']['c8001_5_mc'] 
chfs[2015]['ind_hh']['c_cp'] = chfs[2015]['ind_hh']['c8001_6_mc'] 
chfs[2015]['ind_hh']['c_aud'] = chfs[2015]['ind_hh']['c8001_7_mc'] 
chfs[2015]['ind_hh']['c_wh'] = chfs[2015]['ind_hh']['c8001_8_mc'] 
chfs[2015]['ind_hh']['c_fur'] = chfs[2015]['ind_hh']['c8001_9_mc'] 
chfs[2015]['ind_hh']['c_sr'] = chfs[2015]['ind_hh']['c8001_10_mc'] 
chfs[2015]['ind_hh']['c_mi'] = chfs[2015]['ind_hh']['c8001_12_mc'] 
chfs[2015]['ind_hh']['c_mobi'] = chfs[2015]['ind_hh']['c8001_13_mc'] 

#c8004_7_mc: 没有其它资产 c8004_5_mc: 金银首饰
chfs[2015]['ind_hh']['c_oa'] = chfs[2015]['ind_hh']['c8004_7_mc'].map({1: 0}).fillna(1)
chfs[2015]['ind_hh']['c_je'] = chfs[2015]['ind_hh']['c8004_5_mc']

#吃饭消费：g1001_imp 月*12
chfs[2015]['ind_hh']['g_eat'] = chfs[2015]['ind_hh']['g1001_imp']*12
#日用品支出：g1006_imp 月*12
chfs[2015]['ind_hh']['g_dn'] = chfs[2015]['ind_hh']['g1006_imp']*12
#雇佣保姆小时工等支出：g1007_imp 月*12
chfs[2015]['ind_hh']['g_hw'] = chfs[2015]['ind_hh']['g1007_imp']*12
#本地交通费：g1008_imp 月*12
chfs[2015]['ind_hh']['g_tp'] = chfs[2015]['ind_hh']['g1008_imp']*12
#电话网络通信费：g1009_imp 月*12
chfs[2015]['ind_hh']['g_comm'] = chfs[2015]['ind_hh']['g1009_imp']*12
#文化娱乐：g1010_imp 月*12
chfs[2015]['ind_hh']['g_et'] = chfs[2015]['ind_hh']['g1010_imp']*12
#水电燃气+暖气：g1005_imp月*12+g1013_imp年
chfs[2015]['ind_hh'] = chfs[2015]['ind_hh'].assign(g_sdrm=chfs[2015]['ind_hh']['g1005_imp']*12+chfs[2015]['ind_hh']['g1013_imp'])
#买衣服：g1011_imp 年
chfs[2015]['ind_hh']['g_clo'] = chfs[2015]['ind_hh']['g1011_imp']
#教育培训支出：g1016_imp 年
chfs[2015]['ind_hh']['g_edu'] = chfs[2015]['ind_hh']['g1016_imp']
#旅游探亲支出：g1018_imp 年
chfs[2015]['ind_hh']['g_trav'] = chfs[2015]['ind_hh']['g1018_imp']
#医疗保健支出：g1019_imp 年
chfs[2015]['ind_hh']['g_med'] = chfs[2015]['ind_hh']['g1019b_imp']

#d1105_imp: 活期存款 d2104_imp: 定期存款
chfs[2015]['ind_hh']['d_dd'] = chfs[2015]['ind_hh']['d1105_imp']
chfs[2015]['ind_hh']['d_fd'] = chfs[2015]['ind_hh']['d2104_imp']
#债券：d4103_(1~5)_imp
for i in range(1,6):
    chfs[2015]['ind_hh']['d_bond_'+str(i)] = chfs[2015]['ind_hh']['d4103_'+str(i)+'_imp']
#债券总
chfs[2015]['ind_hh'] = chfs[2015]['ind_hh'].assign(d_bond=chfs[2015]['ind_hh']['d_bond_1']+chfs[2015]['ind_hh']['d_bond_2']+chfs[2015]['ind_hh']['d_bond_3']+chfs[2015]['ind_hh']['d_bond_4']+chfs[2015]['ind_hh']['d_bond_5'])
#银行理财/互联网理财：d7106a_imp
chfs[2015]['ind_hh']['d_fm'] = chfs[2015]['ind_hh']['d7106a_imp']
#黄金/贵金属：d9100aa_imp
chfs[2015]['ind_hh']['d_gold'] = chfs[2015]['ind_hh']['d9100aa_imp']
#现金：k1101_imp
chfs[2015]['ind_hh']['d_cash'] = chfs[2015]['ind_hh']['k1101_imp']

chfs[2015]['ind_hh']['total_income'] = chfs[2015]['ind_hh']['total_income_imp']

chfs[2015]['ind_hh']['year'] = 2015

## 输出表

In [24]:
chfs[2015]['ind_hh'][['hhid','year','age','male','female', \
                      'el_non','el_pri','el_jun','el_sen','el_ts','el_jc','el_bac','el_mas','el_phd', \
                      'c_car','c_car_n','c_car2', \
                      'c_cam','c_tv','c_wm','c_ref','c_ac','c_cp','c_aud','c_wh','c_fur','c_sr','c_mi','c_mobi', \
                      'c_oa','c_je', \
                      'd_dd','d_fd','d_bond','d_fm','d_gold','d_cash', \
                      'g_eat','g_sdrm','g_dn','g_hw','g_tp','g_comm','g_et', \
                      'g_clo','g_edu','g_trav','g_med', \
                      'mem_cnt','labor_inc','labor_inc_sum','labor_inc_mem_cnt','total_income']]
#                       'mem_cnt','labor_inc','labor_inc_sum','labor_inc_mem_cnt','total_income']].to_csv("./chfs2015/final.csv",index=False)

# 2017年数据

In [10]:
# #预处理 dta转csv
# pd.read_stata("./chfs2017/ind.dta").fillna(0).to_csv("./chfs2017/ind.csv",index=False)
# pd.read_stata("./chfs2017/hh.dta").fillna(0).to_csv("./chfs2017/hh.csv",index=False)
# pd.read_stata("./chfs2017/master.dta").fillna(0).to_csv("./chfs2017/master.csv",index=False)

## 读取数据

In [25]:
chfs[2017]['ind'] = cd.read_csv("./chfs2017/ind.csv").drop(['hhid_2011','hhid_2013','hhid_2015','hhid_2017','track'], axis=1)
chfs[2017]['ind'].shape

(127012, 703)

In [26]:
chfs[2017]['hh'] = cd.read_csv("./chfs2017/hh.csv").drop(['hhid_2011','hhid_2013','hhid_2015','hhid_2017'], axis=1)
chfs[2017]['hh'].shape

(40011, 2527)

In [27]:
chfs[2017]['m'] = cd.read_csv("./chfs2017/master.csv").drop(['hhid_2011','hhid_2013','hhid_2015','hhid_2017'], axis=1)
chfs[2017]['m'].shape

(127012, 49)

## 拼接三个表

In [28]:
chfs[2017]['ind_hh'] = cd.merge(chfs[2017]['ind'], chfs[2017]['hh'], how='left', on='hhid')
chfs[2017]['ind_hh'] = cd.merge(chfs[2017]['ind_hh'], chfs[2017]['m'], how='left', on=['hhid','pline'])
del chfs[2017]['ind']
del chfs[2017]['hh']
del chfs[2017]['m']
chfs[2017]['ind_hh'].shape

(127012, 3276)

## 计算表中未有的参数值

In [29]:
chfs[2017]['ind_hh'] = chfs[2017]['ind_hh'].assign(age=2017-chfs[2017]['ind_hh'].a2005) #计算age

dict_mem_cnt = chfs[2017]['ind_hh'].groupby('hhid').size().to_pandas().to_dict() #计算家庭成员个数
chfs[2017]['ind_hh']['mem_cnt'] = chfs[2017]['ind_hh']['hhid'].map(dict_mem_cnt)

chfs[2017]['ind_hh']['labor_inc'] = chfs[2017]['ind_hh']['a3136_imp']+chfs[2017]['ind_hh']['a3136a_imp']+chfs[2017]['ind_hh']['a3136b_imp']+chfs[2017]['ind_hh']['a3125_imp']
dict_labor_inc = chfs[2017]['ind_hh'].groupby('hhid')['labor_inc'].agg('sum').to_pandas().to_dict() #计算家庭总labor收入
chfs[2017]['ind_hh']['labor_inc_sum'] = chfs[2017]['ind_hh']['hhid'].map(dict_labor_inc)

dict_labor_inc_mem_cnt = chfs[2017]['ind_hh'].groupby('hhid')['labor_inc'].count().to_pandas().to_dict() #计算家庭有labor收入的人数
chfs[2017]['ind_hh']['labor_inc_mem_cnt'] = chfs[2017]['ind_hh']['hhid'].map(dict_labor_inc_mem_cnt)

#性别
chfs[2017]['ind_hh']['male'] = chfs[2017]['ind_hh']['a2003'].map({1: 1}).fillna(0) #男
chfs[2017]['ind_hh']['female'] = chfs[2017]['ind_hh']['a2003'].map({2: 1}).fillna(0) #女

#学历
chfs[2017]['ind_hh']['el_non'] = chfs[2017]['ind_hh']['a2012'].map({1: 1}).fillna(0) #没上过学
chfs[2017]['ind_hh']['el_pri'] = chfs[2017]['ind_hh']['a2012'].map({2: 1}).fillna(0) #小学
chfs[2017]['ind_hh']['el_jun'] = chfs[2017]['ind_hh']['a2012'].map({3: 1}).fillna(0) #初中
chfs[2017]['ind_hh']['el_sen'] = chfs[2017]['ind_hh']['a2012'].map({4: 1}).fillna(0) #高中
chfs[2017]['ind_hh']['el_ts'] = chfs[2017]['ind_hh']['a2012'].map({5: 1}).fillna(0) #中专 
chfs[2017]['ind_hh']['el_jc'] = chfs[2017]['ind_hh']['a2012'].map({6: 1}).fillna(0) #大专
chfs[2017]['ind_hh']['el_bac'] = chfs[2017]['ind_hh']['a2012'].map({7: 1}).fillna(0) #本科
chfs[2017]['ind_hh']['el_mas'] = chfs[2017]['ind_hh']['a2012'].map({8: 1}).fillna(0) #硕士
chfs[2017]['ind_hh']['el_phd'] = chfs[2017]['ind_hh']['a2012'].map({9: 1}).fillna(0) #博士

#车
chfs[2017]['ind_hh']['c_car'] = chfs[2017]['ind_hh']['c7002'].map({0: 0}).fillna(1) #汽车，从c7002数量获取
chfs[2017]['ind_hh']['c_car_n'] = chfs[2017]['ind_hh']['c7002'] #汽车数量
chfs[2017]['ind_hh']['c_car2'] = chfs[2017]['ind_hh']['c7058_imp'].map({0: 0}).fillna(1) #拖拉机等


chfs[2017]['ind_hh']['year'] = 2017

## 处理数据column成统一可读性列名

In [30]:
#c8001aa: 手机1、电视2、洗衣机3、冰箱4、空调5、电脑6、音响7、热水器8、家具9、卫星接收器10、乐器11、相机12
chfs[2017]['ind_hh']['c_cam'] = chfs[2017]['ind_hh']['c8001aa_12_mc'] 
chfs[2017]['ind_hh']['c_tv'] = chfs[2017]['ind_hh']['c8001aa_2_mc'] 
chfs[2017]['ind_hh']['c_wm'] = chfs[2017]['ind_hh']['c8001aa_3_mc'] 
chfs[2017]['ind_hh']['c_ref'] = chfs[2017]['ind_hh']['c8001aa_4_mc'] 
chfs[2017]['ind_hh']['c_ac'] = chfs[2017]['ind_hh']['c8001aa_5_mc'] 
chfs[2017]['ind_hh']['c_cp'] = chfs[2017]['ind_hh']['c8001aa_6_mc'] 
chfs[2017]['ind_hh']['c_aud'] = chfs[2017]['ind_hh']['c8001aa_7_mc'] 
chfs[2017]['ind_hh']['c_wh'] = chfs[2017]['ind_hh']['c8001aa_8_mc'] 
chfs[2017]['ind_hh']['c_fur'] = chfs[2017]['ind_hh']['c8001aa_9_mc'] 
chfs[2017]['ind_hh']['c_sr'] = chfs[2017]['ind_hh']['c8001aa_10_mc'] 
chfs[2017]['ind_hh']['c_mi'] = chfs[2017]['ind_hh']['c8001aa_11_mc'] 
chfs[2017]['ind_hh']['c_mobi'] = chfs[2017]['ind_hh']['c8001aa_1_mc'] 

#c8004_7788_mc: 没有其它资产 c8004_5_mc: 金银首饰
chfs[2017]['ind_hh']['c_oa'] = chfs[2017]['ind_hh']['c8004_7788_mc'].map({1: 0}).fillna(1)
chfs[2017]['ind_hh']['c_je'] = chfs[2017]['ind_hh']['c8004_5_mc']

#吃饭消费：g1001_imp 月*12
chfs[2017]['ind_hh']['g_eat'] = chfs[2017]['ind_hh']['g1001_imp']*12
#日用品支出：g1006_imp 月*12
chfs[2017]['ind_hh']['g_dn'] = chfs[2017]['ind_hh']['g1006_imp']*12
#雇佣保姆小时工等支出：g1007_imp 月*12
chfs[2017]['ind_hh']['g_hw'] = chfs[2017]['ind_hh']['g1007_imp']*12
#本地交通费：g1008_imp 月*12
chfs[2017]['ind_hh']['g_tp'] = chfs[2017]['ind_hh']['g1008_imp']*12
#电话网络通信费：g1009_imp 月*12
chfs[2017]['ind_hh']['g_comm'] = chfs[2017]['ind_hh']['g1009_imp']*12
#文化娱乐：g1010_imp 月*12
chfs[2017]['ind_hh']['g_et'] = chfs[2017]['ind_hh']['g1010_imp']*12
#水电燃气+暖气：g1005_imp 月*12
chfs[2017]['ind_hh']['g_sdrm'] = chfs[2017]['ind_hh']['g1005_imp']*12
#买衣服：g1011_imp 年
chfs[2017]['ind_hh']['g_clo'] = chfs[2017]['ind_hh']['g1011_imp']
#教育培训支出：g1016_imp 年
chfs[2017]['ind_hh']['g_edu'] = chfs[2017]['ind_hh']['g1016_imp']
#旅游探亲支出：g1018_imp 年
chfs[2017]['ind_hh']['g_trav'] = chfs[2017]['ind_hh']['g1018_imp']
#仅医疗支出：g1019_imp 年
chfs[2017]['ind_hh']['g_med'] = chfs[2017]['ind_hh']['g1019_imp']

#d1105_imp: 活期存款 d2104_imp: 定期存款
chfs[2017]['ind_hh']['d_dd'] = chfs[2017]['ind_hh']['d1105_imp']
chfs[2017]['ind_hh']['d_fd'] = chfs[2017]['ind_hh']['d2104_imp']
#债券：d4103_(1~5)_imp
for i in range(1,6):
    chfs[2017]['ind_hh']['d_bond_'+str(i)] = chfs[2017]['ind_hh']['d4103_'+str(i)+'_imp']
#债券总
chfs[2017]['ind_hh'] = chfs[2017]['ind_hh'].assign(d_bond=chfs[2017]['ind_hh']['d_bond_1']+chfs[2017]['ind_hh']['d_bond_2']+chfs[2017]['ind_hh']['d_bond_3']+chfs[2017]['ind_hh']['d_bond_4']+chfs[2017]['ind_hh']['d_bond_5'])
#银行理财/互联网理财：d7106h_imp
chfs[2017]['ind_hh']['d_fm'] = chfs[2017]['ind_hh']['d7106h_imp']
#黄金/贵金属：d9103_imp
chfs[2017]['ind_hh']['d_gold'] = chfs[2017]['ind_hh']['d9103_imp']
#现金：k1101_imp
chfs[2017]['ind_hh']['d_cash'] = chfs[2017]['ind_hh']['k1101_imp']

# chfs[2017]['ind_hh']['total_income'] = chfs[2017]['ind_hh']['total_income'] # 2017直接有这列，无需处理

chfs[2017]['ind_hh']['year'] = 2017

## 输出表

In [31]:
chfs[2017]['ind_hh'][['hhid','year','age','male','female', \
                      'el_non','el_pri','el_jun','el_sen','el_ts','el_jc','el_bac','el_mas','el_phd', \
                      'c_car','c_car_n','c_car2', \
                      'c_cam','c_tv','c_wm','c_ref','c_ac','c_cp','c_aud','c_wh','c_fur','c_sr','c_mi','c_mobi', \
                      'c_oa','c_je', \
                      'd_dd','d_fd','d_bond','d_fm','d_gold','d_cash', \
                      'g_eat','g_sdrm','g_dn','g_hw','g_tp','g_comm','g_et', \
                      'g_clo','g_edu','g_trav','g_med', \
                      'mem_cnt','labor_inc','labor_inc_sum','labor_inc_mem_cnt','total_income']]
#                       'mem_cnt','labor_inc','labor_inc_sum','labor_inc_mem_cnt','total_income']].to_csv("./chfs2017/final.csv",index=False)

# 2019年数据

In [11]:
# #预处理 dta转csv
# pd.read_stata("./chfs2019/ind.dta").fillna(0).to_csv("./chfs2019/ind.csv",index=False)
# pd.read_stata("./chfs2019/hh.dta").fillna(0).to_csv("./chfs2019/hh.csv",index=False)
# pd.read_stata("./chfs2019/master.dta").fillna(0).to_csv("./chfs2019/master.csv",index=False)

## 读取数据

In [32]:
chfs[2019]['ind'] = cd.read_csv("./chfs2019/ind.csv")
chfs[2019]['ind'].shape

(107008, 423)

In [33]:
chfs[2019]['hh'] = cd.read_csv("./chfs2019/hh.csv")
chfs[2019]['hh'].shape

(34643, 2656)

In [34]:
chfs[2019]['m'] = cd.read_csv("./chfs2019/master.csv")
chfs[2019]['m'].shape

(107008, 54)

## 拼接三个表

In [35]:
chfs[2019]['ind_hh'] = cd.merge(chfs[2019]['ind'], chfs[2019]['hh'], how='left', on='hhid')
chfs[2019]['ind_hh'] = cd.merge(chfs[2019]['ind_hh'], chfs[2019]['m'], how='left', on=['hhid','pline'])
del chfs[2019]['ind']
del chfs[2019]['hh']
del chfs[2019]['m']
chfs[2019]['ind_hh'].shape

(107008, 3130)

## 计算表中未有的参数值

In [36]:
chfs[2019]['ind_hh'] = chfs[2019]['ind_hh'].assign(age=2019-chfs[2019]['ind_hh'].a2005) #计算age

dict_mem_cnt = chfs[2019]['ind_hh'].groupby('hhid').size().to_pandas().to_dict() #计算家庭成员个数
chfs[2019]['ind_hh']['mem_cnt'] = chfs[2019]['ind_hh']['hhid'].map(dict_mem_cnt)

chfs[2019]['ind_hh']['labor_inc'] = chfs[2019]['ind_hh']['a3136aa_imp']+chfs[2019]['ind_hh']['a3136_imp']+chfs[2019]['ind_hh']['a3136a_imp']+chfs[2019]['ind_hh']['a3136b_imp']+chfs[2019]['ind_hh']['a3171_imp']
dict_labor_inc = chfs[2019]['ind_hh'].groupby('hhid')['labor_inc'].agg('sum').to_pandas().to_dict() #计算家庭总labor收入
chfs[2019]['ind_hh']['labor_inc_sum'] = chfs[2019]['ind_hh']['hhid'].map(dict_labor_inc)

dict_labor_inc_mem_cnt = chfs[2019]['ind_hh'].groupby('hhid')['labor_inc'].count().to_pandas().to_dict() #计算家庭有labor收入的人数
chfs[2019]['ind_hh']['labor_inc_mem_cnt'] = chfs[2019]['ind_hh']['hhid'].map(dict_labor_inc_mem_cnt)

#性别
chfs[2019]['ind_hh']['male'] = chfs[2019]['ind_hh']['a2003'].map({1: 1}).fillna(0) #男
chfs[2019]['ind_hh']['female'] = chfs[2019]['ind_hh']['a2003'].map({2: 1}).fillna(0) #女

#学历
chfs[2019]['ind_hh']['el_non'] = chfs[2019]['ind_hh']['a2012'].map({1: 1}).fillna(0) #没上过学
chfs[2019]['ind_hh']['el_pri'] = chfs[2019]['ind_hh']['a2012'].map({2: 1}).fillna(0) #小学
chfs[2019]['ind_hh']['el_jun'] = chfs[2019]['ind_hh']['a2012'].map({3: 1}).fillna(0) #初中
chfs[2019]['ind_hh']['el_sen'] = chfs[2019]['ind_hh']['a2012'].map({4: 1}).fillna(0) #高中
chfs[2019]['ind_hh']['el_ts'] = chfs[2019]['ind_hh']['a2012'].map({5: 1}).fillna(0) #中专 
chfs[2019]['ind_hh']['el_jc'] = chfs[2019]['ind_hh']['a2012'].map({6: 1}).fillna(0) #大专
chfs[2019]['ind_hh']['el_bac'] = chfs[2019]['ind_hh']['a2012'].map({7: 1}).fillna(0) #本科
chfs[2019]['ind_hh']['el_mas'] = chfs[2019]['ind_hh']['a2012'].map({8: 1}).fillna(0) #硕士
chfs[2019]['ind_hh']['el_phd'] = chfs[2019]['ind_hh']['a2012'].map({9: 1}).fillna(0) #博士

#车
chfs[2019]['ind_hh']['c_car'] = chfs[2019]['ind_hh']['c7002'].map({0: 0}).fillna(1) #汽车，从c7002数量获取
chfs[2019]['ind_hh']['c_car_n'] = chfs[2019]['ind_hh']['c7002'] #汽车数量
chfs[2019]['ind_hh']['c_car2'] = chfs[2019]['ind_hh']['c7058_imp'].map({0: 0}).fillna(1) #拖拉机等


chfs[2019]['ind_hh']['year'] = 2019

## 处理数据column成统一可读性列名

In [37]:
#c8001: 手机1、电视2、洗衣机烘干机3、冰箱4、空调5、电脑6、音响12、热水器等厨卫大件19、家具9、卫星接收器7777、高档乐器健身器材11、相机12
chfs[2019]['ind_hh']['c_cam'] = chfs[2019]['ind_hh']['c8001ab_12_mc'] 
chfs[2019]['ind_hh']['c_tv'] = chfs[2019]['ind_hh']['c8001ab_2_mc'] 
chfs[2019]['ind_hh']['c_wm'] = chfs[2019]['ind_hh']['c8001ab_3_mc'] 
chfs[2019]['ind_hh']['c_ref'] = chfs[2019]['ind_hh']['c8001ab_4_mc'] 
chfs[2019]['ind_hh']['c_ac'] = chfs[2019]['ind_hh']['c8001ab_5_mc'] 
chfs[2019]['ind_hh']['c_cp'] = chfs[2019]['ind_hh']['c8001ab_6_mc'] 
chfs[2019]['ind_hh']['c_aud'] = chfs[2019]['ind_hh']['c8001ab_12_mc'] 
chfs[2019]['ind_hh']['c_wh'] = chfs[2019]['ind_hh']['c8001ab_19_mc'] #之前都是热水器，这里是热水器等厨卫大件
chfs[2019]['ind_hh']['c_fur'] = chfs[2019]['ind_hh']['c8001ab_9_mc'] 
chfs[2019]['ind_hh']['c_sr'] = chfs[2019]['ind_hh']['c8001ab_7777_mc'] 
chfs[2019]['ind_hh']['c_mi'] = chfs[2019]['ind_hh']['c8001ab_11_mc'] 
chfs[2019]['ind_hh']['c_mobi'] = chfs[2019]['ind_hh']['c8001ab_1_mc'] 

#c8004a_7788_mc: 没有其它资产 c8004a_1_mc: 金银首饰
chfs[2019]['ind_hh']['c_oa'] = chfs[2019]['ind_hh']['c8004a_7788_mc'].map({1: 0}).fillna(1)
chfs[2019]['ind_hh']['c_je'] = chfs[2019]['ind_hh']['c8004a_1_mc']

#吃饭消费：g1001_imp 月*12
chfs[2019]['ind_hh']['g_eat'] = chfs[2019]['ind_hh']['g1001_imp']*12
#日用品支出：g1006_imp 月*12
chfs[2019]['ind_hh']['g_dn'] = chfs[2019]['ind_hh']['g1006_imp']*12
#雇佣保姆小时工等支出：g1007_imp 月*12
chfs[2019]['ind_hh']['g_hw'] = chfs[2019]['ind_hh']['g1007_imp']*12
#本地交通费：g1008_imp 月*12
chfs[2019]['ind_hh']['g_tp'] = chfs[2019]['ind_hh']['g1008_imp']*12
#电话网络通信费：g1009_imp 月*12
chfs[2019]['ind_hh']['g_comm'] = chfs[2019]['ind_hh']['g1009_imp']*12
#文化娱乐：g1010_imp 年
chfs[2019]['ind_hh']['g_et'] = chfs[2019]['ind_hh']['g1010_imp'] #2019年问的是年
#水电燃气+暖气：g1005_imp 月*12
chfs[2019]['ind_hh']['g_sdrm'] = chfs[2019]['ind_hh']['g1005_imp']*12
#买衣服：g1011_imp 年
chfs[2019]['ind_hh']['g_clo'] = chfs[2019]['ind_hh']['g1011_imp']
#教育培训支出：g1016_imp 年
chfs[2019]['ind_hh']['g_edu'] = chfs[2019]['ind_hh']['g1016_imp']
#旅游探亲支出：g1018_imp 年
chfs[2019]['ind_hh']['g_trav'] = chfs[2019]['ind_hh']['g1018_imp']
#仅保健支出：g1020b_imp 年
chfs[2019]['ind_hh']['g_med'] = chfs[2019]['ind_hh']['g1020b_imp']

#d1105_imp: 活期存款 d2104_imp: 定期存款
chfs[2019]['ind_hh']['d_dd'] = chfs[2019]['ind_hh']['d1105_imp']
chfs[2019]['ind_hh']['d_fd'] = chfs[2019]['ind_hh']['d2104_imp']
#债券总：d4103_imp
chfs[2019]['ind_hh']['d_bond'] = chfs[2019]['ind_hh']['d4103_imp']
#银行理财/互联网理财：d7106hb_imp
chfs[2019]['ind_hh']['d_fm'] = chfs[2019]['ind_hh']['d7106hb_imp']
#黄金/贵金属：d9103_imp
chfs[2019]['ind_hh']['d_gold'] = chfs[2019]['ind_hh']['d9103_imp']
#现金：k1101_imp+d7106ha_imp余额宝
chfs[2019]['ind_hh']['d_cash'] = chfs[2019]['ind_hh']['k1101_imp']+chfs[2019]['ind_hh']['d7106ha_imp']

# chfs[2019]['ind_hh']['total_income'] = chfs[2019]['ind_hh']['total_income'] # 2019直接有这列，无需处理

chfs[2019]['ind_hh']['year'] = 2019

## 输出表

In [38]:
chfs[2019]['ind_hh'][['hhid','year','age','male','female', \
                      'el_non','el_pri','el_jun','el_sen','el_ts','el_jc','el_bac','el_mas','el_phd', \
                      'c_car','c_car_n','c_car2', \
                      'c_cam','c_tv','c_wm','c_ref','c_ac','c_cp','c_aud','c_wh','c_fur','c_sr','c_mi','c_mobi', \
                      'c_oa','c_je', \
                      'd_dd','d_fd','d_bond','d_fm','d_gold','d_cash', \
                      'g_eat','g_sdrm','g_dn','g_hw','g_tp','g_comm','g_et', \
                      'g_clo','g_edu','g_trav','g_med', \
                      'mem_cnt','labor_inc','labor_inc_sum','labor_inc_mem_cnt','total_income']]
#                       'mem_cnt','labor_inc','labor_inc_sum','labor_inc_mem_cnt','total_income']].to_csv("./chfs2019/final.csv",index=False)